# Look at runs with no nudges 
This removes effect of changes to pitch and yaw.
Gives clear picture of how electron params effect coherent edge

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import statistics
pd.set_option('display.max_columns',500)

In [ ]:
df_2020 = pd.read_csv("csv_data/merged/Spring2020_with_up_time.csv")
df_2023 = pd.read_csv("csv_data/merged/Spring2023_with_up_time.csv")
df_2025 = pd.read_csv("csv_data/merged/Spring2025_with_up_time.csv")
print(df_2020.columns)

In [ ]:
df_2020.dropna(inplace=True)
df_2023.dropna(inplace=True)
df_2025.dropna(inplace=True)

We only want good beam times, so require the beam has been "up" for at least 30 seconds

In [ ]:
def select_data(df_in):
    df_out = df_in[df_in['BeamUpTime']>30]
    return df_out

    
df_2020 = select_data(df_2020)
df_2023 = select_data(df_2023)
df_2025 = select_data(df_2025)

# plot time series of electron and photon beam params vs time

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import statistics
pd.set_option('display.max_columns',500)

# plot time series of coherent edge, and electron beam x, y, current, energy
def plot_run(df_in, runNumber, set_ylims = False, ymin=8600, ymax=8620):

    df_run = df_in[(df_in['RunNumber']==runNumber)&(df_in['DAQ:STATUS']==2)&(df_in['EBEAM:CURRENT']>50)]
    df_run.reset_index(inplace=True)

    if df_run.empty:
        return
    radiator_name = df_run['RADIATOR:NAME'].mode()[0].strip('deg')
     # we want only runs with diamond radiator, so skip all with amorphous
    if 'Al' in radiator_name:
        return
    
    print("plotting for run ",runNumber)

    fig, (ax1, ax2, ax3,ax4,ax5) = plt.subplots(5,1,sharex=True)
    
    # plot coherent edge energy vs time
    ax1.plot(df_run['TimeStamp'],df_run['CBREM:ENERGY'],color='b')

    if set_ylims:
        ax1.set_ylim(ymin,ymax)
    ax1.yaxis.set_minor_locator(ticker.MultipleLocator(5))


    # plot electron beam x, y, current, and energy vs time
    ax2.plot(df_run['TimeStamp'],df_run['EBEAM:X'],color='r')
    ax2.set_ylabel('X (mm)')

    ax3.plot(df_run['TimeStamp'],df_run['EBEAM:Y'],color='g')
    ax3.set_ylabel('Y (mm)')
    plt.xlabel("Time Stamp (s)")

    ax4.plot(df_run['TimeStamp'],df_run['EBEAM:CURRENT'])
    ax4.set_ylabel("I (nA)")

    ax5.plot(df_run['TimeStamp'],df_run['EBEAM:ENERGY'])
    ax5.set_ylabel("$E(e^-) [MeV]$")
 
    plane_value = df_run.loc[0,'CBREM:PLANE']

    plane_str = 'PARA'
    if plane_value==2:
        plane_str = "PERP"
    elif plane_value==0:
        plane_str = "Undefined"


    titleStr = 'Run '+str(runNumber)+' '+radiator_name+plane_str
    ax1.set_title(titleStr)
    ax1.set_ylabel("$E_\gamma$ [MeV]")
    ax1.legend()

    plt.savefig("plots/noNudge/plot_run_"+str(runNumber)+".pdf")
 
    plt.show()

    plt.clf()


# Plot summary correlations for each run period and orientation

In [ ]:
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression 

# run period is either Spring20, Spring23, or Spring25
def summary_correlations(df_in, run_period):

    if df_in.empty:
        print('input data frame empty, exit')
        exit()
    roll_0_90 = 0
    roll_45_135 = 0

    if run_period == 'Spring20':
        roll_0_90 = -10.5
        roll_45_135 = 34.5
    elif run_period == 'Spring23':
        roll_0_90 = 162
        roll_45_135 = -153
    elif run_period == 'Spring25':
        roll_0_90 = -16.6
        roll_45_135 = 28.4
    else:
        print("run_period",run_period,'not defined')
        exit()

    # data frame for times with DAQ on, beam up at least 30 seconds, separated by orientation
    df_0_90_PERP = df_in[(df_in['DAQ:STATUS']==2)&(df_in['GONI:ROLL']==roll_0_90)&(df_in['CBREM:PLANE']==2)]
    df_0_90_PARA = df_in[(df_in['DAQ:STATUS']==2)&(df_in['GONI:ROLL']==roll_0_90)&(df_in['CBREM:PLANE']==1)]

    df_45_135_PERP = df_in[(df_in['DAQ:STATUS']==2)&(df_in['GONI:ROLL']==roll_45_135)&(df_in['CBREM:PLANE']==2)]
    df_45_135_PARA = df_in[(df_in['DAQ:STATUS']==2)&(df_in['GONI:ROLL']==roll_45_135)&(df_in['CBREM:PLANE']==1)]

    df_list = [df_0_90_PERP,df_0_90_PARA,df_45_135_PERP,df_45_135_PARA]
    title_list = ["0/90 PERP","0/90 PARA","45/135 PERP","45/135 PARA"]
    outStr = ["0_90_PERP","0_90_PARA","45_135_PERP","45_135_PARA"]
    #plt, (ax1, ax2, ax3, ax4) = plt.subplots(4,1)
    #for method in ['pearson','spearman']:
    for method in ['pearson']:
        for i in range(len(df_list)):

            ax = plt.subplot(1,1,1)
            df_tmp = df_list[i].copy()
            df_tmp = df_tmp[['CBREM:ENERGY_LAG_ADJUSTED','EBEAM:X',"EBEAM:Y",'EBEAM:CURRENT','EBEAM:ENERGY']]
            if df_tmp.empty:
                print('dataframe is empty for ',title_list[i])
            #corr_matrix = np.array(df_tmp.corr(method=method)['CBREM:ENERGY']).reshape(-1,1)[1:]
            corr_matrix = np.array(df_tmp.corr(method=method)['CBREM:ENERGY_LAG_ADJUSTED']).reshape(-1,1)[1:]
            var_list = ['Electron X','Electron Y','Current','$E_e$']

            print(corr_matrix)

            sns.heatmap(corr_matrix,annot=True)
            ax.set_yticklabels(var_list)
            ax.set_xticklabels(['CBREM:ENERGY_LAG_ADJUSTED'])
            plt.title(run_period+" "+title_list[i]+" "+method.title()+" Correlations")
            plt.savefig("plots/noNudge/"+method+"_corr_plot_"+outStr[i]+'_'+run_period+'.pdf')
            plt.show()

In [ ]:
summary_correlations(df_2020, 'Spring20')

In [ ]:
summary_correlations(df_2023, 'Spring23')

In [ ]:
summary_correlations(df_2025, 'Spring25')

Plot x and y of beam for each run period

In [ ]:
def plot_xy(df_in,title):
    plt.scatter(df_in['EBEAM:X'],df_in['EBEAM:Y'],alpha=0.1)
    plt.title(title)
    plt.xlabel("Electron X [mm]")
    plt.ylabel("Electron Y [mm]")

In [ ]:
plot_xy(df_2020,'Spring 2020 y vs x')

In [ ]:
plot_xy(df_2023,'Spring 2023 y vs x')

In [ ]:
plot_xy(df_2025,'Spring 2025 y vs x')

In [ ]:
spring20_runs = df_2020['RunNumber'].unique()
spring23_runs = df_2023['RunNumber'].unique()
spring25_runs = df_2025['RunNumber'].unique()

In [ ]:
print(len(spring20_runs))
# time series for each spring20 run
# uncomment to make time series plots for every spring20 run
#for runNum in spring20_runs:
#    plot_run(df_2020,runNum)

In [ ]:
print(len(spring23_runs))
# time series for each spring23 run
# uncomment to make time series plots for every spring23 run
#for runNum in spring23_runs:
#    plot_run(df_2023,runNum)

In [ ]:
print(len(spring25_runs))
# time series for each spring25 run
# uncomment to make time series plots for every spring25 run
#for runNum in spring25_runs:
#    plot_run(df_2025,runNum)

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib as mpl


def plot_corrs(df_in, runNumber):
    df_tmp = df_in[(df_in['RunNumber']==runNumber)&(df_in['EBEAM:CURRENT']>50)]
    df_tmp.reset_index(inplace=True)

    plane_value = df_tmp.loc[0,'CBREM:PLANE']
    radiator_name = df_tmp['RADIATOR:NAME'].mode()[0].strip('deg')

    # want only diamond runs, so remove amorphous
    if 'Al' in radiator_name:
        return

    plane_str = 'PARA'
    if plane_value==2:
        plane_str = "PERP"
    elif plane_value==0:
        plane_str = "Undefined"
    

    fig = plt.figure(figsize=(8,8))
    gs = gridspec.GridSpec(4,2,figure=fig,hspace=0.4)

    ax1 = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[1,0])
    ax3 = fig.add_subplot(gs[2,0])
    ax4 = fig.add_subplot(gs[3,0])
    ax5 = fig.add_subplot(gs[:,1])

    ax1.scatter(df_tmp['EBEAM:X'],df_tmp['CBREM:ENERGY_LAG_ADJUSTED'],alpha=0.2,rasterized=True)
    ax1.set_xlabel("Electron X [mm]")
    ax1.set_ylabel("$E_\gamma$ [MeV]")
    ax1.set_yscale('log')
    ax1.set_title("Run "+str(runNumber)+" "+radiator_name+plane_str)

    
    ax2.scatter(df_tmp['EBEAM:Y'],df_tmp['CBREM:ENERGY_LAG_ADJUSTED'],alpha=0.2,rasterized=True)
    ax2.set_xlabel("Electron Y [mm]")
    ax2.set_yscale('log')
    ax2.set_ylabel("$E_\gamma$ [MeV]")


    ax3.scatter(df_tmp['EBEAM:CURRENT'],df_tmp['CBREM:ENERGY_LAG_ADJUSTED'],alpha=0.2,rasterized=True)
    ax3.set_xlabel("Current [nA]")
    ax3.set_yscale('log')
    ax3.set_ylabel("$E_\gamma$ [MeV]")

    ax4.scatter(df_tmp['EBEAM:ENERGY'],df_tmp['CBREM:ENERGY_LAG_ADJUSTED'],alpha=0.2,rasterized=True)
    ax4.xaxis.set_major_locator(plt.MaxNLocator(4))
    ax4.set_xlabel("Electron Energy [MeV]")
    ax4.set_yscale('log')
    ax4.set_ylabel("$E_\gamma$ [MeV]")



    # just grab variables of interest, in the correct order 
    # keep coherent edge energy at beginning, but drop after 
    df_viz = df_tmp[['CBREM:ENERGY_LAG_ADJUSTED','EBEAM:X',"EBEAM:Y",'EBEAM:CURRENT','EBEAM:ENERGY']]
    corr_matrix = np.array(df_viz.corr()['CBREM:ENERGY_LAG_ADJUSTED']).reshape(-1,1)[1:]

    var_list = ['Electron X',"Electron Y",'Current','Electron Energy']

    print(corr_matrix)

    sns.heatmap(corr_matrix,ax=ax5,annot=True)
    ax5.set_yticklabels(var_list)
    ax5.set_xticklabels(['$E_\gamma$'])
    ax5.set_title("Correlation Map")
 
    plt.savefig("plots/noNudge/run_"+str(runNumber)+"_correlations.pdf")
    plt.show()
    plt.clf()

In [ ]:
print(len(spring20_runs))
# uncomment to make time series plots for every spring20 run
#for runVal in spring20_runs:
#    plot_corrs(df_2020, runVal)

In [ ]:
print(len(spring23_runs))
# uncomment to make time series plots for every spring20 run
#for runVal in spring23_runs:
#    plot_corrs(df_2023, runVal)

In [ ]:
print(len(spring25_runs))
# uncomment to make time series plots for every spring25 run
#for runVal in spring25_runs:
#    plot_corrs(df_2025, runVal)

After running commented blocks above, we manually pick out some of the most striking runs for visualizing how electron beam params effect coherent edge

# Runs with large spread in X

In [ ]:
# run with dramatic x spread
plot_corrs(df_2020,72676)
plot_run(df_2020,72676)

In [ ]:
plot_corrs(df_2023,120351)
plot_run(df_2023,120351)

# Runs with large spread in y

In [ ]:
# run with dramatic y spread 
plot_corrs(df_2020,72122)
plot_run(df_2020,72122)

In [ ]:
plot_corrs(df_2023,120366)
plot_run(df_2023,120355)

In [ ]:
plot_corrs(df_2025,131440)
plot_run(df_2025,131440)

# Runs with large spread in electron beam energy

In [ ]:
# run with a spread in electron beam energy 
plot_corrs(df_2020,72412)
plot_run(df_2020,72412)

In [ ]:
plot_corrs(df_2023,120371)
plot_run(df_2023,120371)

In [ ]:
plot_corrs(df_2025,131454)
plot_run(df_2025,131454)

# Runs with strange electron beam current behavior

In [ ]:
# periodic behavior in electron beam current, no impact on coherent edge
plot_corrs(df_2020,72656)
plot_run(df_2020,72656,True,8590,8610)

# Pair plots for each run period. 
They take a long time to run since they are reading in a large amount of data

In [ ]:
df_2020_pair = df_2020[(df_2020['CBREM:ENERGY_LAG_ADJUSTED']>8000)&(df_2020['EBEAM:ENERGY']>1000)]
sns.pairplot(df_2020_pair[['CBREM:ENERGY_LAG_ADJUSTED','EBEAM:X','EBEAM:Y','EBEAM:CURRENT','EBEAM:ENERGY']],plot_kws={'alpha':0.1})

In [ ]:
df_2023_pair = df_2023[(df_2023['CBREM:ENERGY']>5000)&(df_2023['EBEAM:ENERGY']>2000)]
sns.pairplot(df_2023_pair[['CBREM:ENERGY','EBEAM:X','EBEAM:Y','EBEAM:CURRENT','EBEAM:ENERGY']],plot_kws={'alpha':0.1})

In [ ]:
df_2025_pair = df_2025[(df_2025['EBEAM:ENERGY']>3000)&(df_2025['EBEAM:ENERGY']<11750)]
sns.pairplot(df_2025_pair[['CBREM:ENERGY','EBEAM:X','EBEAM:Y','EBEAM:CURRENT','EBEAM:ENERGY']])